In [218]:
def IsEquivalentOrdering(seq1,seq2,noncomm):
    """check whether seq1 and seq2 are equivalent via noncomm"""
    assert len(seq1)==len(seq2),"size of objects do not match"
    if len(seq1)==0:return True
    e = seq1[0] # the minimal element
    if e not in seq2:return False
    pos = seq2.index(e) # position of the first element e in seq2
    if len(noncomm[e].intersection(seq2[:pos])): 
        return False
    return IsEquivalentOrdering(seq1[1:],seq2[:pos]+seq2[pos+1:],noncomm)

In [111]:
def noncommutative_roots(root,positive_roots):
    '''positive roots which not commutates with the given root'''
    return {r for r in positive_roots if root+r in positive_roots}

In [108]:
def Word2Roots(word,simple_roots,gens):
    r'''\Phi^+ \intersection w^{-1}\Phi^-
    simple_roots: dict
    gens: list or tuple
    '''
    word = word[::-1] # reverse the expression
    one = gens[0]/gens[0] # the unit of weyl group
    word_roots = [simple_roots[i] for i in word] # word to simple roots
    word_re = [gens[i-1] for i in word] # word to simple reflections
    actions = [prod(word_re[:i],one) for i in range(len(word))]
    return [r.weyl_action(a) for a,r in zip(actions,word_roots)]

In [87]:
def prod(l,one=1):
    '''列表乘积'''
    for i in l:one*=i
    return one

In [261]:
"""1. basic invariables"""
T = CartanType("A3")
R = RootSystem(T)
V = R.root_space()
W = WeylGroup(T)

gens = W.gens()
delta = V.simple_roots() # simple roots
positive = list(V.positive_roots())
noncomm = {r:noncommutative_roots(r,positive) for r in positive}

w0 = W.long_element() # the longest element
w0_words = w0.reduced_words()
w0_seqs = [Word2Roots(w,delta,gens) for w in w0_words]
w0_word = w0.reduced_word() # reduced word of the longest element
w_seq = Word2Roots(w0_word,delta,gens)

'''2. different expression may lead to the same pbw basis'''
res = []
for i,w1 in enumerate(w0_seqs):
    res.append([])
    for j,w2 in enumerate(w0_seqs):
        if i>=j:
            res[-1].append(True if i==j else res[j][i])
            continue
        res[-1].append(IsEquivalentOrdering(w1,w2,noncomm))
print([sum(line) for line in res])

"""3. example(A3): different orderings that lead to the same pbw basis"""
w1 = w0_words[8]
w2 = w0_words[11]
s1 = Word2Roots(w1,delta,gens)
s2 = Word2Roots(w2,delta,gens)
table([w1,w2,s1,s2])

[1, 1]
[[True, False], [False, True]]


In [266]:
"""4. a sepecial ordering (via Dynkin diagram)"""
# T = CartanType("C4")
# print(DynkinDiagram(T))
# R = RootSystem(T)
# V = R.root_space()
# W = WeylGroup(T)
# gens = W.gens()
# delta = V.simple_roots() # simple roots
# positive = list(V.positive_roots())
# noncomm = {r:noncommutative_roots(r,positive) for r in positive}
# w0 = W.long_element() # the longest element
# w0_words = w0.reduced_words()
print("number of different expression of w0:",len(w0_words))
w0_seqs = [Word2Roots(w,delta,gens) for w in w0_words]


#### Type A ####
# type A2
# inds = [(1,),(1,2),(2,)]

# type A3
# inds = [(1,),(1,2),(1,2,3),(2,),(2,3),(3,)]

# type A4
# inds = [(1,),(1,2),(1,2,3),(1,2,3,4),(2,),(2,3),(2,3,4),(3,),(3,4),(4,)]

# standard = [sum(delta[i] for i in ind) for ind in inds]

#### Type C ####
# type C2
# head = [delta[1]]

# type C3
# head = [delta[1],delta[2],delta[1]+delta[2]]

# type C4
inds = [(1,),(1,2),(1,2,3),(2,),(2,3),(3,)]
head = [sum(delta[i] for i in ind) for ind in inds]

standard = head + [r for r in positive if r not in head]


res = []
for i,seq in enumerate(w0_seqs):
    if IsEquivalentOrdering(standard,seq,noncomm):
        res.append(w0_words[i])
#         res.append(seq)
print(T)
table(res)

['C', 4]


,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


In [282]:
res2 = res.copy()
res2.sort()

In [288]:
res2[-1]

[4, 3, 4, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 1, 2, 1]

In [286]:
l = list(range(4,0,-1))+list(range(4,0,-1))+list(range(4,0,-1))+list(range(4,0,-1))

In [287]:
l in res

True

In [227]:
def alternate(list1,list2):
    '''combine two lists alternately'''
    new = []
    for i,j in zip(list1,list2):
        new.extend([i,j])
    return new

In [201]:
pattern = r"(...)\1"
re.sub(pattern,r"3","hhha")

'hhha'

In [192]:
re.sub?
f=lambda s:re.sub('alpha','a',str(s))

In [139]:
# print(len(w0.reduced_words()))
# G = w0.reduced_word_graph()
# G.plot3d()